In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

#Create FLAG PARAMETER

In [0]:
dbutils.widgets.text('incremental_flag','0')

In [0]:
incremantal_flag= dbutils.widgets.get('incremental_flag')
print(incremantal_flag)

#Creating Dimesion Model

###Fetch relative columns

In [0]:
df=spark.sql('''
select distinct (Model_Id)as model_id,
 Model_Catergory from 
 parquet.`abfss://silver@carkrishdatalake.dfs.core.windows.net/Carsales/`''')

In [0]:
 df.display()

the below statement always returns empty table because of dim_model_key as 0 basically it just returns the schema of the table we want to create 

In [0]:
%sql
select 1 as dim_model_key,Model_Id,Model_Catergory from parquet.`abfss://silver@carkrishdatalake.dfs.core.windows.net/Carsales/`
where 1=0;

lets just assign the above statement to the variable using spark 
and this one is also used for initial 
only if table not exists

###dim_model_sink initial and incremental

In [0]:
if spark.catalog.tableExists('cars_catalog.gold.dim_model'):
    # Use the correct column name as suggested by the error message
    df_sink = spark.sql('''
              select dim_model_key, Model_ID, Model_Catergory
              from cars_catalog.gold.dim_model
              ''')

else:
    df_sink = spark.sql('''
                      select 1 as dim_model_key, Model_Id, Model_Catergory
                      from parquet.`abfss://silver@carkrishdatalake.dfs.core.windows.net/Carsales`
                      where 1=0
                      ''')


In [0]:
df_sink.display()

###BRINGING already existing Records

###Filtering new records and old records 

In [0]:
df_filter= df.join(df_sink,df['Model_Id']==df_sink['Model_Id'],'left').select (df['Model_Id'],df['Model_Catergory'],df_sink['dim_model_key'])

In [0]:
df_filter.display()

**df_filter old**

In [0]:
df_filter_old= df_filter.filter(col('dim_model_key').isNotNull())

In [0]:
df_filter_old.display()

**df_filter_new**


In [0]:
df_filter_new= df_filter.filter(col('dim_model_key').isNull()).select (df['Model_ID'],df['Model_Catergory'] )

In [0]:
df_filter_new.display()

### CREATE SURROGATE KEY


****Fetch tha max Surrogate key from exsisting table****

In [0]:
if incremantal_flag=='0':
    max_value=1
else:
    max_value_df = spark.sql('''
                          select max(dim_model_key) from cars_catalog.gold.dim_model''')
    max_value=max_value_df.collect()[0][0]+1

***CREATE SURROGATE key collumn and add surrogate key***

In [0]:
df_filter_new= df_filter_new.withColumn('dim_model_key',max_value+monotonically_increasing_id())

In [0]:
df_filter_new.display()

### create FInal df -df filter old+df filter neew

In [0]:
df_final=df_filter_new.union(df_filter_old)

The result will be the same because we don't have any initial load in the initial run but later in the rounds it will be different.

In [0]:
df_final.display()

#SCD type-1 (upsert/slowing changing dimension type-1)

In [0]:
from delta.tables import DeltaTable 

In [0]:
spark.conf.set(
    "fs.azure.account.key.carkrishndatalake.dfs.core.windows.net",
    "<Storage-account Access key"
)


In [0]:
if spark.catalog.tableExists('cars_catalog.gold.dim_model'):
    # If the Delta table exists, perform the merge operation
    delta_table = DeltaTable.forPath(spark, 'abfss://gold@carkrishdatalake.dfs.core.windows.net/dim_table')
    
    delta_table.alias('trg').merge(df_final.alias('src'), "trg.dim_model_key = src.dim_model_key") \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute()

else:
    # If the table doesn't exist, create it by writing as a Delta table
    df_final.write.format('delta') \
        .mode('overwrite') \
        .option('path', 'abfss://gold@carkrishdatalake.dfs.core.windows.net/dim_table') \
        .saveAsTable('cars_catalog.gold.dim_model')



In [0]:
%sql
select * from cars_catalog.gold.dim_model